In [2]:
%pip install camel-ai[model_platforms,huggingface,web_tools] unstructured python-dotenv dune-client pandas numpy tabulate

Note: you may need to restart the kernel to use updated packages.


# Importing environment variables

In [3]:
%load_ext dotenv
%dotenv

import os
import pandas as pd
import numpy as np

# Preparing memecoin data

## Connecting to dune client

In [4]:
from dune_client.client import DuneClient
dune = DuneClient(os.environ['DUNE_API_KEY'])

## Querying 24H Memecoin Growth Holder

In [5]:
from dune_client.query import QueryBase
from dune_client.types import QueryParameter

query_90d = QueryBase(
    name="90d_memecoin_growth_holder",
    query_id=4948190
)

query_30d = QueryBase(
    name="30d_memecoin_growth_holder",
    query_id=4948159
)

# 7-day data
query_7d = QueryBase(
    name="7d_memecoin_growth_holder",
    query_id=4947030
)

# 24-hour data
query_24h = QueryBase(
    name="24h_memecoin_growth_holder",
    query_id=4947022
)

In [6]:
response_90d = dune.run_query_dataframe(query_90d)
response_30d = dune.run_query_dataframe(query_30d)
response_7d = dune.run_query_dataframe(query_7d)
response_24h = dune.run_query_dataframe(query_24h)

2025-04-09 00:09:32,257 INFO dune_client.api.base executing 4948190 on medium cluster
2025-04-09 00:09:33,166 INFO dune_client.api.base waiting for query execution 01JRBB6HQ4NHAVR3GT4PKM5FJH to complete: ExecutionState.PENDING (queue position: 34)
2025-04-09 00:09:34,368 INFO dune_client.api.base waiting for query execution 01JRBB6HQ4NHAVR3GT4PKM5FJH to complete: ExecutionState.PENDING (queue position: 34)
2025-04-09 00:09:35,549 INFO dune_client.api.base waiting for query execution 01JRBB6HQ4NHAVR3GT4PKM5FJH to complete: ExecutionState.EXECUTING
2025-04-09 00:09:36,734 INFO dune_client.api.base waiting for query execution 01JRBB6HQ4NHAVR3GT4PKM5FJH to complete: ExecutionState.EXECUTING
2025-04-09 00:09:37,922 INFO dune_client.api.base waiting for query execution 01JRBB6HQ4NHAVR3GT4PKM5FJH to complete: ExecutionState.EXECUTING
2025-04-09 00:09:39,116 INFO dune_client.api.base waiting for query execution 01JRBB6HQ4NHAVR3GT4PKM5FJH to complete: ExecutionState.EXECUTING
2025-04-09 00:09:4

In [7]:
response_90d.head()

,coin,optimized_relative_strength,holder_now,three_month_change
0,KIZUNA,3.113389,8492.0,-0.009102
1,DOGE(ETH),2.972326,34083.0,0.168947
2,HAPPY,2.969645,113772.0,-0.086924
3,LUNA,2.914792,303941.0,0.761609
4,TRUMP,2.907812,51140.0,0.254169


In [8]:
## Implementing softmax weighting for timeframes
def softmax(x):
    """Compute softmax values for each set of scores in x."""
    e_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
    return e_x / e_x.sum()

# Add after the softmax function definition

def detect_market_condition(rs_data):
    """
    Detect market condition based on relative strength patterns
    Returns: 'volatile', 'trending', or 'stable'
    """
    # Get the top 20 coins by 24h RS for analyzing recent movement
    top_recent = rs_data.sort_values('rs_24h', ascending=False).head(20)
    
    # Calculate the ratio between short-term and long-term RS
    top_recent['short_long_ratio'] = (top_recent['rs_24h'] + top_recent['rs_7d']) / \
                                     (top_recent['rs_90d'] + top_recent['rs_30d'] + 0.001)  # Avoid division by zero
    
    # Calculate metrics to determine market state
    avg_ratio = top_recent['short_long_ratio'].mean()
    ratio_std = top_recent['short_long_ratio'].std()
    
    # Determine market condition
    if ratio_std > 2.0:  # High variation between coins
        return 'volatile'
    elif avg_ratio > 3.0:  # Short-term RS much higher than long-term
        return 'trending'
    else:
        return 'stable'

# Add after the detect_market_condition function

def dynamic_weight_adjustment(rs_data, base_weights=None):
    """
    Dynamically adjust timeframe weights based on market conditions.
    
    Args:
        rs_data: DataFrame with RS values across timeframes
        base_weights: Optional base weights to modify (if None, uses default importance)
        
    Returns:
        adjusted_weights: np.array of adjusted weights after softmax
        market_condition: string indicating detected market condition
    """
    # Detect current market condition
    market_condition = detect_market_condition(rs_data)
    
    # Define importance adjustments based on market condition
    if market_condition == 'volatile':
        # In volatile markets, heavily favor very recent data (24h, 7d)
        importance = np.array([0.5, 1.0, 2.5, 3.5])  # 90d, 30d, 7d, 24h
        print("Volatile market detected - Increasing weight on recent timeframes")
        
    elif market_condition == 'trending':
        # In trending markets, balance recent and medium-term data
        importance = np.array([0.8, 1.5, 2.2, 2.8])  # 90d, 30d, 7d, 24h
        print("Trending market detected - Balanced weighting with emphasis on recent data")
        
    else:  # stable
        # In stable markets, give more weight to longer-term data
        importance = np.array([1.5, 1.8, 1.6, 1.7])  # 90d, 30d, 7d, 24h
        print("Stable market detected - More balanced weighting across timeframes")
    
    # Apply softmax to get normalized weights
    adjusted_weights = softmax(importance)
    
    return adjusted_weights, market_condition

def combine_timeframes_with_softmax(df_90d, df_30d, df_7d, df_24h, use_dynamic_weights=True):
    """Combine data from different time frames using softmax-weighted RS values."""
    # Merge all dataframes on 'coin' column
    combined_df = df_90d[['coin', 'optimized_relative_strength']].copy()
    combined_df.rename(columns={'optimized_relative_strength': 'rs_90d'}, inplace=True)
    
    # Merge with 30d data
    df_30d_subset = df_30d[['coin', 'optimized_relative_strength']].copy()
    df_30d_subset.rename(columns={'optimized_relative_strength': 'rs_30d'}, inplace=True)
    combined_df = combined_df.merge(df_30d_subset, on='coin', how='outer')
    
    # Merge with 7d data
    df_7d_subset = df_7d[['coin', 'optimized_relative_strength']].copy()
    df_7d_subset.rename(columns={'optimized_relative_strength': 'rs_7d'}, inplace=True)
    combined_df = combined_df.merge(df_7d_subset, on='coin', how='outer')
    
    # Merge with 24h data
    df_24h_subset = df_24h[['coin', 'optimized_relative_strength']].copy()
    df_24h_subset.rename(columns={'optimized_relative_strength': 'rs_24h'}, inplace=True)
    combined_df = combined_df.merge(df_24h_subset, on='coin', how='outer')
    
    # Fill NaN values with 0
    combined_df.fillna(0, inplace=True)
    
    if use_dynamic_weights:
        # Use dynamic weight adjustment based on market conditions
        weights, market_condition = dynamic_weight_adjustment(combined_df)
        combined_df['market_condition'] = market_condition
    else:
        # Use static weights (original implementation)
        importance = np.array([1.0, 1.5, 2.0, 2.5])  # Increasing importance: 90d, 30d, 7d, 24h
        weights = softmax(importance)
        combined_df['market_condition'] = 'static'
    
    # Calculate weighted RS
    combined_df['weighted_rs'] = (
        weights[0] * combined_df['rs_90d'] +
        weights[1] * combined_df['rs_30d'] +
        weights[2] * combined_df['rs_7d'] +
        weights[3] * combined_df['rs_24h']
    )
    
    # Sort by weighted RS
    combined_df.sort_values(by='weighted_rs', ascending=False, inplace=True)
    
    return combined_df, weights

# Combine data from all timeframes with dynamic softmax weights
combined_data, weights = combine_timeframes_with_softmax(
    response_90d, response_30d, response_7d, response_24h, 
    use_dynamic_weights=True  # Set to True to use dynamic weights
)

# Print the weights used
print(f"Timeframe weights (90d, 30d, 7d, 24h): {weights}")
print(f"Market condition: {combined_data['market_condition'].iloc[0]}")

# Create a function to prepare combined data for LLM
def prepare_combined_df_for_llm(df):
    # Select relevant columns for display
    relevant_df = df[['coin', 'rs_90d', 'rs_30d', 'rs_7d', 'rs_24h', 'weighted_rs']].head(15).copy()
    
    # Round values for better display
    for col in relevant_df.columns[1:]:
        relevant_df[col] = relevant_df[col].round(4)
    
    # Convert to markdown table
    markdown_table = relevant_df.to_markdown(index=False)
    return markdown_table

Volatile market detected - Increasing weight on recent timeframes
Timeframe weights (90d, 30d, 7d, 24h): [0.03319688 0.0547324  0.2452936  0.66677713]
Market condition: volatile


In [9]:
def compare_weight_strategies():
    """Compare different weighting strategies and their effect on top coin rankings"""
    # Get rankings with dynamic weights
    dynamic_df, dynamic_weights = combine_timeframes_with_softmax(
        response_90d, response_30d, response_7d, response_24h, 
        use_dynamic_weights=True
    )
    dynamic_top10 = dynamic_df[['coin', 'weighted_rs', 'market_condition']].head(10)
    dynamic_top10['rank'] = range(1, 11)
    dynamic_top10 = dynamic_top10[['rank', 'coin', 'weighted_rs', 'market_condition']]
    
    # Get rankings with static weights
    static_df, static_weights = combine_timeframes_with_softmax(
        response_90d, response_30d, response_7d, response_24h, 
        use_dynamic_weights=False
    )
    static_top10 = static_df[['coin', 'weighted_rs']].head(10)
    static_top10['rank'] = range(1, 11)
    static_top10 = static_top10[['rank', 'coin', 'weighted_rs']]
    
    # Create manual volatility scenario (double 24h importance)
    volatile_importance = np.array([0.5, 1.0, 2.0, 4.0])
    volatile_weights = softmax(volatile_importance)
    static_df['volatile_weighted_rs'] = (
        volatile_weights[0] * static_df['rs_90d'] +
        volatile_weights[1] * static_df['rs_30d'] +
        volatile_weights[2] * static_df['rs_7d'] +
        volatile_weights[3] * static_df['rs_24h']
    )
    static_df.sort_values(by='volatile_weighted_rs', ascending=False, inplace=True)
    volatile_top10 = static_df[['coin', 'volatile_weighted_rs']].head(10)
    volatile_top10['rank'] = range(1, 11)
    volatile_top10 = volatile_top10[['rank', 'coin', 'volatile_weighted_rs']]
    volatile_top10.rename(columns={'volatile_weighted_rs': 'weighted_rs'}, inplace=True)
    
    print("=== Dynamic Weights (Market Condition: {}) ===".format(dynamic_df['market_condition'].iloc[0]))
    print(f"Weights (90d, 30d, 7d, 24h): {dynamic_weights}")
    print(dynamic_top10)
    print("\n=== Static Weights (Original) ===")
    print(f"Weights (90d, 30d, 7d, 24h): {static_weights}")
    print(static_top10)
    print("\n=== High Volatility Scenario ===")
    print(f"Weights (90d, 30d, 7d, 24h): {volatile_weights}")
    print(volatile_top10)
    
    # Calculate changes in ranking
    diff_count = sum(1 for c1, c2 in zip(dynamic_top10['coin'], static_top10['coin']) if c1 != c2)
    print(f"\nDifference between dynamic and static: {diff_count} changes in top 10")
    
    # Return the different top coins
    dynamic_set = set(dynamic_top10['coin'])
    static_set = set(static_top10['coin'])
    volatile_set = set(volatile_top10['coin'])
    
    print("\nCoins uniquely in dynamic top 10:", dynamic_set - static_set)
    print("Coins uniquely in static top 10:", static_set - dynamic_set)
    print("Coins uniquely in volatile top 10:", volatile_set - static_set)
    
    return dynamic_top10, static_top10, volatile_top10

# Run the comparison
dynamic_top10, static_top10, volatile_top10 = compare_weight_strategies()

Volatile market detected - Increasing weight on recent timeframes
=== Dynamic Weights (Market Condition: volatile) ===
Weights (90d, 30d, 7d, 24h): [0.03319688 0.0547324  0.2452936  0.66677713]
    rank      coin  weighted_rs market_condition
27     1      LUCE    23.419287         volatile
47     2      PORK    20.146116         volatile
35     3       MOG    18.095165         volatile
50     4  RETARDIO    16.908027         volatile
63     5       USA    15.310045         volatile
3      6     BONGO    14.230764         volatile
49     7      PUPS    14.095668         volatile
5      8    BONSAI    10.203870         volatile
8      9    CHEEMS     7.195719         volatile
34    10     MOCHI     6.505288         volatile

=== Static Weights (Original) ===
Weights (90d, 30d, 7d, 24h): [0.10153632 0.1674051  0.27600434 0.45505423]
    rank      coin  weighted_rs
27     1      LUCE    19.727245
47     2      PORK    13.484161
35     3       MOG    12.079112
50     4  RETARDIO    11.4829

In [10]:
def generate_cot_analysis(coin_data, weights):
    """Generate a structured chain-of-thought analysis template for memecoin data."""
    timeframe_weights = {
        "90d": weights[0],
        "30d": weights[1],
        "7d": weights[2],
        "24h": weights[3]
    }
    
    cot_template = f"""
When analyzing memecoin performance, follow these reasoning steps:

1. TIMEFRAME WEIGHTS:
   - 90-day weight: {timeframe_weights['90d']:.4f} (longer-term trend)
   - 30-day weight: {timeframe_weights['30d']:.4f} (medium-term trend)
   - 7-day weight: {timeframe_weights['7d']:.4f} (short-term trend)
   - 24-hour weight: {timeframe_weights['24h']:.4f} (immediate momentum)

2. REASONING PROCESS:
   For each coin:
   - Examine its performance across all timeframes
   - Multiply each RS value by its corresponding weight
   - Calculate the contribution of each timeframe to the final score
   - Identify which timeframes drive the coin's ranking

3. CONSISTENCY ANALYSIS:
   - Coins with high RS values across multiple timeframes show consistent strength
   - Coins with recent spikes (high 24h/7d but low 90d/30d) show momentum but may lack longevity
   - Coins with high long-term metrics but low recent performance may be losing momentum

4. COMPARATIVE FRAMEWORK:
   - Compare coins against each other to establish relative performance
   - Place findings in context of the overall memecoin market
   - Evaluate which factors are most predictive for each coin's performance
"""
    
    return cot_template

# Creating the agent

## Creating the model

In [11]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

# Define the model, here in this case we use Gemini
model = ModelFactory.create(
    model_platform=ModelPlatformType.GEMINI,
    model_type=ModelType.GEMINI_2_0_FLASH,
)

## Creating the Chat Agent

In [12]:
from camel.agents import ChatAgent

# Generate the CoT analysis template
cot_analysis = generate_cot_analysis(combined_data, weights)

# Prepare data
multi_timeframe_data = prepare_combined_df_for_llm(combined_data)

# Get market condition
market_condition = combined_data['market_condition'].iloc[0]

system_message = f"""You are a cryptocurrency analysis expert specializing in memecoins.
You have access to the following data about memecoins across multiple timeframes:

{multi_timeframe_data}

The data includes:
- 90-day relative strength (rs_90d)
- 30-day relative strength (rs_30d)
- 7-day relative strength (rs_7d)
- 24-hour relative strength (rs_24h)
- Weighted relative strength that emphasizes recent performance

Current market condition: {market_condition.upper()}
Based on this market condition, the weighting algorithm has {
    "increased the importance of recent data (24h, 7d)" if market_condition == 'volatile' else
    "balanced recent and medium-term data" if market_condition == 'trending' else
    "given more weight to longer-term performance"
}.

{cot_analysis}

When providing analysis, always:
1. Start with your INITIAL OBSERVATIONS of the data
2. Consider the current market condition ({market_condition}) in your analysis
3. THINK STEP-BY-STEP through your analysis of each coin's performance
4. SHOW YOUR CALCULATIONS for how each timeframe contributes to the weighted score
5. EXPLAIN YOUR REASONING for why certain coins rank higher than others
6. CONCLUDE with a clear summary and answer to the question

Format your reasoning in distinct sections so your analytical process is transparent.
Only use the data provided to you. If asked for information outside this dataset,
acknowledge the limitations of your data.
"""

agent = ChatAgent(
    system_message=system_message,
    model=model
)

## Getting a sample response

In [13]:
# Generate a sample analysis with full chain-of-thought reasoning
detailed_question = """Analyze the top 10 memecoins in the dataset. 
For each coin:
1. Break down how each timeframe contributes to its weighted score
2. Calculate the percentage contribution from each timeframe
3. Explain whether its strength comes from consistent performance or recent momentum
4. Compare it to the other top coins
"""

response = agent.step(detailed_question)
print(response.msgs[0].content)

2025-04-09 00:12:22,113 INFO httpx HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
2025-04-09 00:12:22,121 INFO camel.agents.chat_agent Model gemini-2.0-flash-exp, index 0, processed these messages: [{'role': 'system', 'content': "You are a cryptocurrency analysis expert specializing in memecoins.\nYou have access to the following data about memecoins across multiple timeframes:\n\n| coin     |   rs_90d |   rs_30d |   rs_7d |   rs_24h |   weighted_rs |\n|:---------|---------:|---------:|--------:|---------:|--------------:|\n| LUCE     |   1.1351 |  13.2063 | 17.8388 |  27.42   |       23.4193 |\n| PORK     |   1.7297 |  -1.2968 | -2.1428 |  31.0228 |       20.1461 |\n| MOG      |   2.6825 |  -2.1395 | -1.8749 |  27.8701 |       18.0952 |\n| RETARDIO |   2.5889 |  -1.9356 | -0.0802 |  25.4173 |       16.908  |\n| USA      |   1.7917 |  -3.3867 | -2.6851 |  24.1379 |       15.31   |\n| BONGO    |   2.8472 |  -2.375  |  0     

Okay, I will analyze the top 10 memecoins in the dataset based on their weighted relative strength, considering the volatile market conditions and the increased importance of recent data.

**Initial Observations:**

The top 10 coins show positive weighted relative strength, indicating they've generally outperformed other memecoins recently. LUCE and PORK have the highest weighted scores, suggesting strong recent performance. Several coins (MOG, RETARDIO, USA, BONGO, PUPS) show negative 30-day and 7-day RS but high 24-hour RS, implying a recent surge in momentum.

**Memecoin Analysis:**

I will now analyze each of the top 10 memecoins, breaking down their weighted scores and assessing the consistency of their performance.

**1. LUCE:**

*   **Timeframe Contributions:**
    *   90-day: 1.1351 * 0.0332 = 0.0377
    *   30-day: 13.2063 * 0.0547 = 0.7224
    *   7-day: 17.8388 * 0.2453 = 4.3758
    *   24-hour: 27.42 * 0.6668 = 18.2701
*   **Percentage Contribution:**
    *   90-day: (0.037